In [ ]:
from dask.distributed import Client
from dask_kubernetes.operator import KubeCluster
from src.cloud_manager import VMManager

In [ ]:
cluster: KubeCluster = KubeCluster(name="mycluster",
                                   image='hermelinkluntjes/thesis:test',
                                   n_workers=0)
client = Client(cluster)
client

In [ ]:
worker_group_config = {
   "name": "highmem",
   "n_workers": 2,
   "resources":{
      "requests": 
         {"memory": "6Gi",
          "cpu": "3"}
      }
}
cluster.add_worker_group(**worker_group_config)

In [ ]:
cluster.scale(2, worker_group="highmem")
# Dont scale down while cluster scales up -> makes for errors
client.wait_for_workers(2)

In [ ]:
with VMManager(cluster) as vmmanager:
    print(vmmanager.cluster)